# Поиск аномалий (система алертов)

## Описание проекта

Написать систему алертов для приложения. Система должна с периодичностью каждые 15 минут проверять ключевые метрики, такие как активные пользователи в ленте/мессенджере, просмотры, лайки, CTR, количество отправленных сообщений.   
В случае обнаружения аномального значения в телеграм чат должен отправиться алерт - сообщение со следующей информацией: метрика, ее значение, величина отклонения. В сообщение можно добавить дополнительную информацию, которая поможет при исследовании причин возникновения аномалии. Это может быть график, ссылки на дашборд/чар в BI-системе.   
Автоматизировать систему алертов с помощью Airflow. 

## Данные

Таблица feed_actions - данные по ленте новостей:
 - user_id - уникальный номер пользователя
 - post_id - уникальный номер поста
 - action - лайк или просмотр
 - time - время события
 - gender - пол пользователя
 - age - возраст пользователя
 - country - страна пользователя
 - city - город пользователя
 - os - операционная система устройства пользователя
 - sourse - источник привлечения пользователя (organic - поисковик, ads - реклама)
 - exp_group - номер экспериментальной группы
 
Таблица message_actions - данные по сервису обмена сообщениями:
 - user_id - уникальный номер пользователя
 - reciever_id - уникальный номер получетеля
 - time - время события
 - source - источник привлечения пользователя (organic - поисковик, ads - реклама)
 - exp_group - номер экспериментальной группы
 - gender - пол пользователя
 - age - возраст пользователя
 - country - страна пользователя
 - city - город пользователя
 - os - операционная система устройства пользователя

## Результат

Для определения алертов был выбран метод межквартильных размахов. Данный метод удобен для поиска аномалий в realtime.   
Для этого построили доверительный интервал, выход за границы которого будет говорить об аномальности наблюдения. Границы интервала вычислили с помощью квартилей. Коэффициент *a*, определяющий ширину интервала, и количество 15-минуток (_n_) выбрали практическим способом.   
В случае обнаружения алерта в телеграм отправляется сообщение со значением метрики, величиной отклонения, ссылка на общий дашборд и ссылка на график конкретной метрики. Так же сразу отправляется график, отображающий поведение метрики в сравнении с верхней и нижней границами.


## Скрипт определения аномалий и отправки отчета

In [ ]:
# coding=utf-8

from datetime import datetime, timedelta
import pandas as pd
import pandahouse as ph
import matplotlib.pyplot as plt
import seaborn as sns
import io
import sys
import os
import telegram
import numpy as np

from airflow.decorators import dag, task
from airflow.operators.python import get_current_context

connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20220820'
}

default_args = {
    'owner': 'e-varshavskaja-10',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2022, 8, 25), 
}

schedule_interval = '15 * * * *'
my_token = '**********:_________________________'
bot = telegram.Bot(token=my_token)


q1 = '''
        SELECT 
               toStartOfFifteenMinutes(time) as ts,
               toDate(time) as date,
               formatDateTime(ts, '%R') as hm,
               count(distinct user_id) as users,
               SUM(action='view') as views,
               SUM(action='like') as likes,
               likes/views as ctr
        FROM simulator_20220820.feed_actions
        WHERE ts >= today() - 1 and ts < toStartOfFifteenMinutes(now())
        GROUP BY ts, date, hm
        ORDER BY ts
    '''
q2 = '''
    SELECT 
           toStartOfFifteenMinutes(time) as ts,
           toDate(time) as date,
           formatDateTime(ts, '%R') as hm,
           count(distinct user_id) as users,
           count(user_id) as msg_sent
    FROM simulator_20220820.message_actions
    WHERE ts >= today() - 1 and ts < toStartOfFifteenMinutes(now())
    GROUP BY ts, date, hm
    ORDER BY ts 
    '''
    
feed = ph.read_clickhouse(q1, connection=connection)
messages = ph.read_clickhouse(q2, connection=connection)

feed_metrics = ['users','views', 'likes', 'ctr'] 
messages_metrics = ['users', 'msg_sent']

dashboard = 'https://superset.lab.karpov.courses/superset/dashboard/1952/'

feed_plots = {'users': 'http://superset.lab.karpov.courses/r/2465',
              'views': 'http://superset.lab.karpov.courses/r/2466',
              'likes': 'http://superset.lab.karpov.courses/r/2464',
              'ctr': 'http://superset.lab.karpov.courses/r/2467'}

messages_plots = {'users': 'http://superset.lab.karpov.courses/r/2469',
                  'msg_sent' : 'http://superset.lab.karpov.courses/r/2471'}

def check_anamoly(data, metric, a=4, n=5):
    # функция предлагает алгоритм поиска аномалий (межквартильный размах)
    data['q25'] = data[metric].shift(1).rolling(n).quantile(0.25)
    data['q75'] = data[metric].shift(1).rolling(n).quantile(0.75)
    data['iqr'] = data['q75'] - data['q25']
    data['up'] = data['q75'] + a*data['iqr']
    data['low'] = data['q25'] - a*data['iqr']
    
    data['up'] = data['up'].rolling(n, center=True, min_periods=1).mean()
    data['low'] = data['low'].rolling(n, center=True, min_periods=1).mean()
    
    if data[metric].iloc[-1] <  data['low'].iloc[-1] or data[metric].iloc[-1] > data['up'].iloc[-1]:
        is_alert = 1
    else:
        is_alert = 0
    return is_alert, data

def run_alerts(table, metrics, plot, chat=None):
    # система алертов
    
    chat_id = chat or 262611825
    bot = telegram.Bot(token=my_token)
       
    for metric in metrics:
        print(metric)
        data = table[['ts', 'date', 'hm', metric]].copy()
        is_alert, data = check_anamoly(data, metric)
        
        if is_alert == 1:
            msg = '''Метрика {}:\n текущее значение {:.2f},\nотклонение от предыдущего значения {:.2%}\n Дашборд:\n{}\n График алерта:\n{}'''\
            .format(metric, data[metric].iloc[-1], 1-(data[metric].iloc[-1]/data[metric].iloc[-2]), dashboard, plot[metric])
            
            sns.set(rc={'figure.figsize': (16, 10)})
            plt.tight_layout()
            
            ax = sns.lineplot(x=data['ts'], y=data[metric], label='metric')
            ax = sns.lineplot(x=data['ts'], y=data['up'], label='up')
            ax = sns.lineplot(x=data['ts'], y=data['low'], label='low')
            
            for i, label in enumerate(ax.get_xticklabels()):
                if i % 2 == 0:
                    label.set_visible(True)
                else:
                    label.set_visible(False)
                    
            ax.set(xlabel='time')
            ax.set(ylabel=metric)
            
            ax.set_title(metric)
            
            plot_object = io.BytesIO()
            ax.figure.savefig(plot_object)
            plot_object.seek(0)
            plot_object.name = '{}.png'.format(metric)
            plt.close()
            
            bot.sendMessage(chat_id=chat_id, text=msg)
            bot.sendPhoto(chat_id=chat_id, photo=plot_object)

@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def alerts_varshavskaya():
    
    @task
    def feed_alerts():
        run_alerts(feed, feed_metrics, feed_plots)
        
    @task
    def messages_alerts():
        run_alerts(messages, messages_metrics, messages_plots)
    
    feed_alerts()
    messages_alerts()
    
alerts_varshavskaya = alerts_varshavskaya()